In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

def split_scalar(indep_X,dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
    #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
    
    #Feature Scaling
    #from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

def Linear(X_train,y_train,X_test,y_test_original):
    # Fitting K-NN to the Training set
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test_original)
    return r2
    
def svm_linear(X_train,y_train,X_test,y_test_original):
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'linear')
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test_original)
    return r2
    
def svm_NL(X_train,y_train,X_test,y_test_original):
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'rbf')
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test_original)
    return r2
    

def Decision(X_train,y_train,X_test,y_test_original):
    # Fitting K-NN to the Training setC
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test_original)
    return r2
    

def random(X_train,y_train,X_test,y_test_original):
    # Fitting K-NN to the Training set
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
    regressor.fit(X_train, y_train)
    r2=r2_prediction(regressor,X_test,y_test_original)
    return r2
    
def rfeFeature(indep_X,dep_Y,n):
    rfelist=[]
    
    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()
    
    from sklearn.svm import SVR
    SVRl = SVR(kernel = 'linear')
    
    from sklearn.svm import SVR
    #SVRnl = SVR(kernel = 'rbf')
    
    from sklearn.tree import DecisionTreeRegressor
    dec = DecisionTreeRegressor(random_state = 0)
    
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators = 10, random_state = 0)
    
    
    rfemodellist=[lin,SVRl,dec,rf]
    for i in rfemodellist:
        print(i)
        log_rfe = RFE(estimator=i, n_features_to_select=n) #Corrected RFE call
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature=log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)
    return rfelist
    
def rfe_regression(acclog,accsvml,accdes,accrf):
    rfedataframe=pd.DataFrame(index=['Linear','SVC','Random','DecisionTree'],columns=['Linear','SVMl', 'Decision','Random'])
    for number,idex in enumerate(rfedataframe.index):
        rfedataframe.loc[idex, 'Linear'] = acclog[number]
        rfedataframe.loc[idex, 'SVMl'] = accsvml[number]
        rfedataframe.loc[idex, 'Decision'] = accdes[number]
        rfedataframe.loc[idex, 'Random'] = accrf[number]
    return rfedataframe



In [2]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']


In [9]:
rfelist=rfeFeature(indep_X,dep_Y,7)

acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=10, random_state=0)


In [10]:
for i in rfelist:
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)
    r2_lin=Linear(X_train,y_train,X_test,y_test)
    acclin.append(r2_lin)
    
    r2_sl=svm_linear(X_train,y_train,X_test,y_test)
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(X_train,y_train,X_test,y_test)
    accsvmnl.append(r2_NL)
    
    r2_d=Decision(X_train,y_train,X_test,y_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test,y_test)
    accrf.append(r2_r)
    
result=rfe_regression(acclin,accsvml,accdes,accrf)


In [5]:
result
#3

,Linear,SVMl,Decision,Random
Linear,0.441961,0.262153,0.441961,0.441816
SVC,0.441961,0.262153,0.441961,0.441816
Random,0.664893,0.609652,0.965961,0.916304
DecisionTree,0.676174,0.670691,0.933504,0.887256


In [8]:
result
#5

,Linear,SVMl,Decision,Random
Linear,0.620124,0.457136,0.77924,0.780135
SVC,0.604508,0.456871,0.776474,0.776745
Random,0.674403,0.628206,0.696181,0.815538
DecisionTree,0.686361,0.643365,0.836806,0.845303


In [11]:
result
#7

,Linear,SVMl,Decision,Random
Linear,0.622757,0.5373,0.813952,0.814557
SVC,0.623155,0.5296,0.81284,0.8134
Random,0.697704,0.666684,0.913194,0.940972
DecisionTree,0.705879,0.667997,0.797454,0.850957
